### 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 패키지 설치

In [2]:
!pip install openai langchain langchain-google-genai

In [3]:
from pprint import pprint
from typing import Dict, List

from langchain.chains import LLMChain, SequentialChain
# from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel


### OpenAI API key

In [4]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

### Prompt chain 준비
* 서비스할 내용의 프롬프트 체인을 준비합니다.
* 각 프롬프트 체인을 미리 준비해 놓고, 템플릿으로 사용합니다.

In [5]:
P_PATH = "/content/drive/MyDrive/Lecture/LLMs/LLM & Langchain/Langchain examples/Projects/Novel_generation/multi_prompt"
IDEA_P = os.path.join(P_PATH, "extract_idea.txt")
OUTLINE_P = os.path.join(P_PATH, "write_outline.txt")
PLOT_P = os.path.join(P_PATH, "write_plot.txt")
CHAPTER_P = os.path.join(P_PATH, "write_chapter.txt")

### Prompt chain 구현
* `SequentialChain`을 이용해서 여러개의 chain을 연속적으로 구현할 수 있습니다.

In [6]:
class UserRequest(BaseModel):
    genre: str
    characters: List[Dict[str, str]]
    text: str


def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r") as f:
        prompt_template = f.read()

    return prompt_template


def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path),
        ),
        output_key=output_key,
        verbose=True,
    )


def generate_novel(req: UserRequest) -> Dict[str, str]:
    # writer_llm = ChatOpenAI(temperature=0.7, max_tokens=4000, model="gpt-3.5-turbo-0125")
    writer_llm = ChatGoogleGenerativeAI(model="gemini-pro")

    # 아이디어 뽑기 체인 생성
    novel_idea_chain = create_chain(writer_llm, IDEA_P, "novel_idea") # llm, prompt path, "변수명"

    # 아웃라인 작성 체인 생성
    novel_outline_chain = create_chain(
        writer_llm, OUTLINE_P, "novel_outline"
    )

    # 플롯 작성 체인 생성
    novel_plot_chain = create_chain(writer_llm, PLOT_P, "novel_plot")

    # 챕터 작성 체인 생성
    novel_chapter_chain = create_chain(writer_llm, CHAPTER_P, "output")

    preprocess_chain = SequentialChain(
        chains=[
            novel_idea_chain,
            novel_outline_chain,
            novel_plot_chain,
        ],
        input_variables=["genre", "characters", "text"],
        output_variables=["novel_idea", "novel_outline", "novel_plot"],
        verbose=True,
    )

    context = req.dict()
    context = preprocess_chain(context)

    context["novel_chapter"] = []
    for chapter_number in range(1, 9):
        context["chapter_number"] = chapter_number
        context = novel_chapter_chain(context)
        context["novel_chapter"].append(context["output"])

    contents = "\n\n".join(context["novel_chapter"])
    return {"results": contents}

### User prompt 작성
* User가 직접 작성하는 프롬프트를 작성합니다.

In [7]:
user_data = {
    "genre": "판타지",
    "characters": [
        {
            "name": "김철수",
            "role": "주인공"
        },
        {
            "name": "이영희",
            "role": "조연"
        }
    ],
    "text": "날씨가 추워지고 있습니다."
}


* User Prompt를 입력합니다.

In [8]:
request_instance = UserRequest(**user_data)

### Text Generation

In [10]:
res = generate_novel(request_instance)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '김철수', 'role': '주인공'}, {'name': '이영희', 'role': '조연'}]

[참고 텍스트]
날씨가 추워지고 있습니다.

[등장 인물] 과 [참고 텍스트] 를 소재로 새롭고 흥미진진한 판타지 소설 아이디어를 한 문단으로 작성해줘

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '김철수', 'role': '주인공'}, {'name': '이영희', 'role': '조연'}]

[참고 텍스트]
날씨가 추워지고 있습니다.

[아이디어]
추운 겨울 바람이 휘몰아치는 가운데, 주인공 김철수는 요정의 숲 가장자리에 서 있었다. 갑자기, 맑고 달콤한 노래声이 그를 숲 속으로 불러들였다. 그는 조연인 이영희와 함께 신비로운 나무와 빛나는 폭포 사이를 헤매며, 숨겨진 요정의 왕국을 발견했다. 하지만 이 왕국은 암흑의 마법사가 휩쓸고 지배한 상태였고, 철수와 영희는 요정들을 구하고 왕국을 되찾기 위해 위험한 여정을 떠나야 했다. 그들의 여정은 강력한 마법, 용감한 전투, 그리고 탐험과 발견으로 가득할 것이다.

[context]
아웃라인 단계에서는 주요 이벤트와 결말을 고려하세요. 
여기서 중요한 것은, 이 단계에서 구체적인 디테일에 매몰되기보다는 스토리의 큰 그림에 집중하는 것입니다.

[등장 인물] 과 [아이디어] 를 소재로 새롭고 흥미진진한 판타지 소설의 아웃라인을 작성해줘


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '김철수', 'role': '주인공'

In [11]:
print(res['results'])

**Chapter 1: 숲의 부름**

겨울 바람이 울부짖으며 눈송이를 날리며, 젊은 작가 김철수는 요정의 숲 가장자리에 서 있었다. 그는 몇 주 동안 탐험하고 있던 신비한 숲을 바라보고 있었다. 달콤한 노래声が 숲 깊숙한 곳에서 들려왔고, 철수는 마치 누군가 자신을 부르는 것처럼 느꼈다.

"들어가보자." 철수는 중얼거렸다. "아마도 나를 위한 이야기가 기다리고 있을지도 몰라."

철수는 조심스럽게 숲속으로 들어갔다. 나무들은 키가 크고 울창하여 하늘을 가렸고, 땅은 부드러운 이끼와 낙엽으로 뒤덮여 있었다. 달빛이 희미하게 비추는 가운데 철수는 신비로운 나무와 빛나는 폭포 사이를 헤매며 걷기 시작했다.

몇 시간 후, 철수는 숨이 막히는 광경을 목격했다. 밝은 빛에 싸인 거대한 성이 그의 앞에 우뚝 솟아 있었다. 성벽은 금빛으로 반짝였고, 첨탑은 하늘을 향해 뻗어 있었다. 철수는 이곳이 전설에 나오는 요정의 왕국이라는 것을 깨달았다.

철수는 성문으로 다가갔지만, 문은 굳게 닫혀 있었다. 그는 문을 두드렸지만, 아무런 대답이 없었다. 갑자기, 그의 뒤에서 달콤한 목소리가 들렸다.

"누가 문을 두드리고 있나요?"

철수는 뒤돌아보니 젊은 요정 여자가 서 있었다. 그녀는 긴 금발과 반짝이는 녹색 눈을 가지고 있었다.

"안녕하세요." 철수가 말했다. "저는 김철수입니다. 이곳에 들어가고 싶은데요."

"저는 이영희입니다." 요정 여자가 말했다. "이곳은 요정의 왕국입니다. 인간은 들어올 수 없습니다."

"하지만 저는 당신을 도울 수 있어요." 철수가 말했다. "저는 요정의 이야기를 많이 들었고, 저는 당신의 왕국에 위험이 닥쳤다는 것을 알고 있습니다."

이영희는 놀란 표정을 지었다. "위험이요? 무슨 위험인가요?"

"암흑의 마법사가 당신의 왕국을 지배하려고 합니다." 철수가 말했다. "저는 당신을 도울 수 있습니다. 저는 작가입니다. 저는 이야기를 쓸 수 있습니다. 당신의 이야기를 세상에 알릴 수 있습니다."

이영희는 잠시 망설이다가 말했다. "좋습니다